In [11]:
!tleap -f complex.in

-I: Adding /Users/sethmartinez/opt/anaconda3/envs/qcmm/dat/leap/prep to search path.
-I: Adding /Users/sethmartinez/opt/anaconda3/envs/qcmm/dat/leap/lib to search path.
-I: Adding /Users/sethmartinez/opt/anaconda3/envs/qcmm/dat/leap/parm to search path.
-I: Adding /Users/sethmartinez/opt/anaconda3/envs/qcmm/dat/leap/cmd to search path.
-f: Source complex.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./complex.in
----- Source: /Users/sethmartinez/opt/anaconda3/envs/qcmm/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /Users/sethmartinez/opt/anaconda3/envs/qcmm/dat/leap/cmd/leaprc.protein.ff14SB done
Log file: ./leap.log
Loading parameters: /Users/sethmartinez/opt/anaconda3/envs/qcmm/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /Users/sethmartinez/opt/anaconda3/envs/qcmm/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain p